In [1]:
import torch.nn as nn
import torch 
input = torch.randn(3,22,15000)
input.shape

torch.Size([3, 22, 15000])

In [2]:
nn.Conv1d(in_channels=22,out_channels=32,stride=2,kernel_size=2,padding=0)

Conv1d(22, 32, kernel_size=(2,), stride=(2,))

In [3]:
class Block(nn.Module):
    def __init__(self,inplace):
        super().__init__()
        self.conv1=nn.Conv1d(in_channels=inplace,out_channels=32,stride=2,kernel_size=2,padding=0)
        self.conv2=nn.Conv1d(in_channels=inplace,out_channels=32,stride=2,kernel_size=4,padding=1)
        self.conv3=nn.Conv1d(in_channels=inplace,out_channels=32,stride=2,kernel_size=8,padding=3)
        self.relu=nn.ReLU()
    def forward(self,x):
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x= torch.cat([x1,x2,x3],dim=1)
        return x
    

In [4]:
block=Block(22)
out1=block(input)
out1.shape

torch.Size([3, 96, 7500])

In [5]:
block=Block(96)
out2=block(out1)
out2.shape

torch.Size([3, 96, 3750])

In [6]:
block=Block(96)
out3=block(out2)
out3.shape

torch.Size([3, 96, 1875])

In [7]:
gru = nn.GRU(input_size=96,hidden_size=32,batch_first=True) #multilayer gated recurrent unit
x = out3.permute(0,2,1) #(N,L,H) when batch_first=True where N-batchsize L-sequence length and H-input
print(x.shape)
output,hn= gru(x)
print(output.shape)

torch.Size([3, 1875, 96])
torch.Size([3, 1875, 32])


In [8]:
print(hn.shape)

torch.Size([1, 3, 32])


In [9]:
gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
gru1_out, _= gru1(x)
gru2_out,_= gru2(gru1_out)
gru_out = torch.cat([gru1_out,gru2_out],dim=2)

In [10]:
gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
gru3_out,_= gru3(gru_out)

In [11]:
gru_out = torch.cat([gru1_out,gru2_out,gru3_out],dim=2)

In [12]:
gru_out.shape

torch.Size([3, 1875, 96])

In [13]:
gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
gru4_out,_= gru4(gru_out)
gru4_out.shape

torch.Size([3, 1875, 32])

In [14]:
linear = nn.Linear(1875,1)
linear_out = linear(gru_out.permute(0,2,1))
linear_out.shape

torch.Size([3, 96, 1])

In [15]:
gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
gru4_out, hn = gru4(linear_out.permute(0,2,1))
gru4_out.shape

torch.Size([3, 1, 32])

In [16]:
hn.shape

torch.Size([1, 3, 32])

In [38]:
class Chrononet(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1= Block(22)
        self.block2= Block(96)
        self.block3= Block(96)
        self.gru1=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
        self.gru2=nn.GRU(input_size=32,hidden_size=32,batch_first=True)
        self.gru3=nn.GRU(input_size=64,hidden_size=32,batch_first=True)
        self.gru4=nn.GRU(input_size=96,hidden_size=32,batch_first=True)
        self.gru_linear= nn.Linear(1875,1)
        self.flatten = nn.Flatten()
        self.fcl= nn.Linear(32,1)
        self.relu = nn.ReLU()
    def forward(self,x):
        x=self.block1(x)
        x=self.block2(x)
        x=self.block3(x)
        x=x.permute(0,2,1)
        gru1_out, _= self.gru1(x)
        gru2_out, _= self.gru2(gru1_out)
        gru_out = torch.cat([gru1_out,gru2_out], dim=2)
        gru3_out, _= self.gru3(gru_out)
        gru_out = torch.cat([gru1_out,gru2_out,gru3_out], dim=2)

        linear_out = self.relu(self.gru_linear(gru_out.permute(0,2,1)))
        gru4_out, _= self.gru4(linear_out.permute(0,2,1))
        x=self.flatten(gru4_out)
        x = self.fcl(x)
        return x         

In [40]:
model= Chrononet()
output = model(input)

In [ ]:
from glob import glob 
TDD = 
TDC =